<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/lora-bert/source/lora-bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/lora-bert/'):
    os.makedirs('/content/drive/MyDrive/lora-bert/')
output_dir = '/content/drive/MyDrive/lora-bert/'

In [ ]:
!pip install datasets
!pip install evaluate
!pip freeze > /content/drive/MyDrive/lora-bert/requirements.txt

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from transformers import set_seed
set_seed(0)

In [ ]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return {'total_parameters': parameters, 'trainable_parameters': trainable_parameters}

In [ ]:
import csv
def write_results_to_csv(file_name, results):
  with open(file_name, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))
      writer.writeheader()
      for result in results:
        writer.writerow(result)

In [ ]:
def get_model_name(name_and_path):
  return name_and_path.split('/')[1]

# Full Fine-Tuning Implementation



Followed examples from this [Hugging Face NLP course](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api) on how to use the trainer API for fine-tuning.

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, set_seed
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
import evaluate
import time

def ft_hyperparameter_tune(checkpoint, epochs, batch_sizes, learning_rates):
  model_name = checkpoint
  if '/' in checkpoint:
    model_name = get_model_name(checkpoint)

  raw_datasets = load_dataset("glue", "mrpc")
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  def tokenize_function(example):
      return tokenizer(example["sentence1"], example["sentence2"])

  tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  def compute_metrics(eval_preds):
      metric = evaluate.load("glue", "mrpc")
      logits, labels = eval_preds
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)

  ft_results = []
  progress = 0
  iterations = len(batch_sizes)*len(learning_rates)

  # Hyperparameter tuning
  start_time = time.time()
  for size in batch_sizes:
    for rate in learning_rates:
      # Track progress
      progress += 1
      print(f"Progress: {progress}/{iterations}")

      # Initialize result
      result = {"batch_size": size, "learning_rate": rate}

      model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

      # Setup the trainer
      training_args = TrainingArguments(
          "fine-tuning-trainer",
          eval_strategy="epoch",
          per_device_eval_batch_size=size,
          per_device_train_batch_size=size,
          num_train_epochs=epochs,
          learning_rate=rate,
          disable_tqdm=True,
          report_to="none"
      )
      trainer = Trainer(
          model,
          training_args,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["validation"],
          data_collator=data_collator,
          processing_class=tokenizer,
          compute_metrics=compute_metrics
      )

      # Train the model
      ft_results.append(result | trainer.train().metrics | trainer.evaluate() | get_trainable_parameters(model))

  write_results_to_csv(output_dir+model_name+'-ft-results.csv', ft_results)

  end_time = time.time()

  runtime_seconds = end_time - start_time
  runtime_minutes = runtime_seconds / 60

  # Output the best result
  max_ft_result = ft_results[0]
  for _, result in enumerate(ft_results):
    if result['eval_f1'] > max_ft_result['eval_f1']:
      max_ft_result = result
  print(f'\n================ \
          \nTotal Runtime: {runtime_minutes} minutes \
          \nBest Result: \
          \n\tF1={max_ft_result["eval_f1"]} \
          \n\tBatch Size={max_ft_result["batch_size"]} \
          \n\tLearning Rate={max_ft_result["learning_rate"]}'
  )

  return ft_results

# Low-Rank Adaptation (LoRA) Implementation

In [ ]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [ ]:
def configure_lora_model(model, rank, alpha):
  # Replace the query and value linear layers with LoRA layers
  for _, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  # Freeze the pre-trained weights
  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False

In [ ]:
from transformers import Trainer, TrainingArguments, set_seed
import time

def lora_hyperparameter_tune(checkpoint, ranks, alphas, epochs, batch_sizes, learning_rates):
  model_name = checkpoint
  if '/' in checkpoint:
    model_name = get_model_name(checkpoint)

  # Setup model and dataset
  raw_datasets = load_dataset("glue", "mrpc")
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  def tokenize_function(example):
      return tokenizer(example["sentence1"], example["sentence2"])

  tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  def compute_metrics(eval_preds):
      metric = evaluate.load("glue", "mrpc")
      logits, labels = eval_preds
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)

  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

  lora_results = []
  progress = 0
  iterations = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)

  # Hyperparameter tuning
  start_time = time.time()
  for rank in ranks:
    for alpha in alphas:
      for size in batch_sizes:
        for rate in learning_rates:
          # Track progress
          progress += 1
          print(f"Progress: {progress}/{iterations}")

          # Initialize result
          result = {"rank": rank, "alpha": alpha, "batch_size": size, "learning_rate": rate}

          # Configure an existing model with new LoRA layers
          configure_lora_model(model, rank, alpha)

          # Setup the trainer
          training_args = TrainingArguments(
              "lora-trainer",
              eval_strategy="epoch",
              per_device_eval_batch_size=size,
              per_device_train_batch_size=size,
              num_train_epochs=epochs,
              learning_rate=rate,
              disable_tqdm=True,
              report_to="none"
          )
          trainer = Trainer(
              model,
              training_args,
              train_dataset=tokenized_datasets["train"],
              eval_dataset=tokenized_datasets["validation"],
              data_collator=data_collator,
              processing_class=tokenizer,
              compute_metrics=compute_metrics
          )

          # Train the model
          lora_results.append(result | trainer.train().metrics | trainer.evaluate() | get_trainable_parameters(model))

  write_results_to_csv(output_dir+model_name+'-lora-results.csv', lora_results)

  end_time = time.time()

  runtime_seconds = end_time - start_time
  runtime_hours = runtime_seconds / 3600

  # Output the best result
  max_lora_result = lora_results[0]
  for _, result in enumerate(lora_results):
    if result['eval_f1'] > max_lora_result['eval_f1']:
      max_lora_result = result
  print(f'\n================ \
          \nTotal Runtime: {runtime_hours} hours \
          \nBest Result: \
          \n\tF1={max_lora_result["eval_f1"]} \
          \n\tRank={max_lora_result["rank"]} \
          \n\tAlpha={max_lora_result["alpha"]} \
          \n\tBatch Size={max_lora_result["batch_size"]} \
          \n\tLearning Rate={max_lora_result["learning_rate"]}'
  )

  return lora_results

# Run

Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [ ]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

In [ ]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

## BERT Tiny

### Full Fine-Tuning

In [ ]:
_ = ft_hyperparameter_tune("prajjwal1/bert-tiny", epochs, batch_sizes, learning_rates)

Progress: 1/20


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5735582113265991, 'eval_accuracy': 0.7083333333333334, 'eval_f1': 0.8155038759689922, 'eval_runtime': 0.7462, 'eval_samples_per_second': 546.755, 'eval_steps_per_second': 68.344, 'epoch': 1.0}
{'loss': 0.5946, 'grad_norm': 4.331884384155273, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.59549880027771, 'eval_accuracy': 0.7107843137254902, 'eval_f1': 0.8138801261829653, 'eval_runtime': 0.7551, 'eval_samples_per_second': 540.351, 'eval_steps_per_second': 67.544, 'epoch': 2.0}
{'loss': 0.4682, 'grad_norm': 38.01247024536133, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.7907573580741882, 'eval_accuracy': 0.7132352941176471, 'eval_f1': 0.8020304568527918, 'eval_runtime': 0.7908, 'eval_samples_per_second': 515.959, 'eval_steps_per_second': 64.495, 'epoch': 3.0}
{'loss': 0.2993, 'grad_norm': 5.858227252960205, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.2679738562091503}
{'eval_loss': 1.063

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.571905791759491, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8169440242057489, 'eval_runtime': 0.742, 'eval_samples_per_second': 549.881, 'eval_steps_per_second': 68.735, 'epoch': 1.0}
{'loss': 0.5884, 'grad_norm': 4.293759822845459, 'learning_rate': 7.276688453159042e-05, 'epoch': 1.0893246187363834}
{'eval_loss': 0.5654860734939575, 'eval_accuracy': 0.7352941176470589, 'eval_f1': 0.8285714285714286, 'eval_runtime': 0.7268, 'eval_samples_per_second': 561.396, 'eval_steps_per_second': 70.175, 'epoch': 2.0}
{'loss': 0.4993, 'grad_norm': 8.675013542175293, 'learning_rate': 4.5533769063180834e-05, 'epoch': 2.178649237472767}
{'eval_loss': 0.6270501613616943, 'eval_accuracy': 0.7352941176470589, 'eval_f1': 0.8296529968454258, 'eval_runtime': 0.7328, 'eval_samples_per_second': 556.751, 'eval_steps_per_second': 69.594, 'epoch': 3.0}
{'loss': 0.4045, 'grad_norm': 26.807472229003906, 'learning_rate': 1.8300653594771242e-05, 'epoch': 3.2679738562091503}
{'eval_loss': 0.665

...

{'eval_loss': 0.6038169264793396, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.6087, 'eval_samples_per_second': 670.269, 'eval_steps_per_second': 6.571, 'epoch': 1.0}
{'eval_loss': 0.5951903462409973, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8224852071005917, 'eval_runtime': 0.5752, 'eval_samples_per_second': 709.318, 'eval_steps_per_second': 6.954, 'epoch': 2.0}
{'eval_loss': 0.579287052154541, 'eval_accuracy': 0.6911764705882353, 'eval_f1': 0.8090909090909091, 'eval_runtime': 0.591, 'eval_samples_per_second': 690.353, 'eval_steps_per_second': 6.768, 'epoch': 3.0}
{'eval_loss': 0.5748432874679565, 'eval_accuracy': 0.6911764705882353, 'eval_f1': 0.8073394495412844, 'eval_runtime': 0.5887, 'eval_samples_per_second': 693.002, 'eval_steps_per_second': 6.794, 'epoch': 4.0}
{'train_runtime': 6.5158, 'train_samples_per_second': 2251.74, 'train_steps_per_second': 17.803, 'train_loss': 0.5897226004764952, 'epoch': 4.0}
{'eval_loss': 0.574843287

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6207587718963623, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5758, 'eval_samples_per_second': 708.542, 'eval_steps_per_second': 6.946, 'epoch': 1.0}
{'eval_loss': 0.6050003170967102, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5963, 'eval_samples_per_second': 684.19, 'eval_steps_per_second': 6.708, 'epoch': 2.0}
{'eval_loss': 0.5970742106437683, 'eval_accuracy': 0.6887254901960784, 'eval_f1': 0.8145985401459854, 'eval_runtime': 0.5873, 'eval_samples_per_second': 694.747, 'eval_steps_per_second': 6.811, 'epoch': 3.0}
{'eval_loss': 0.5952083468437195, 'eval_accuracy': 0.6936274509803921, 'eval_f1': 0.8169838945827232, 'eval_runtime': 0.5915, 'eval_samples_per_second': 689.78, 'eval_steps_per_second': 6.763, 'epoch': 4.0}
{'train_runtime': 6.5492, 'train_samples_per_second': 2240.291, 'train_steps_per_second': 17.712, 'train_loss': 0.6204661007585197, 'epoch': 4.0}
{'eval_loss': 0.59520834

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6439599990844727, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.6205, 'eval_samples_per_second': 657.537, 'eval_steps_per_second': 6.446, 'epoch': 1.0}
{'eval_loss': 0.6185536980628967, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5924, 'eval_samples_per_second': 688.709, 'eval_steps_per_second': 6.752, 'epoch': 2.0}
{'eval_loss': 0.6116704344749451, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.591, 'eval_samples_per_second': 690.338, 'eval_steps_per_second': 6.768, 'epoch': 3.0}
{'eval_loss': 0.609874427318573, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.6017, 'eval_samples_per_second': 678.114, 'eval_steps_per_second': 6.648, 'epoch': 4.0}
{'train_runtime': 6.6213, 'train_samples_per_second': 2215.88, 'train_steps_per_second': 17.519, 'train_loss': 0.6398460125101024, 'epoch': 4.0}
{'eval_loss': 0.609874427

### LoRA

In [ ]:
_ = lora_hyperparameter_tune("prajjwal1/bert-tiny", ranks, alphas, epochs, batch_sizes, learning_rates)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Progress: 1/500
{'eval_loss': 0.6257390975952148, 'eval_accuracy': 0.6813725490196079, 'eval_f1': 0.8104956268221575, 'eval_runtime': 0.7665, 'eval_samples_per_second': 532.311, 'eval_steps_per_second': 66.539, 'epoch': 1.0}
{'loss': 0.663, 'grad_norm': 0.46347522735595703, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.6210986375808716, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.7382, 'eval_samples_per_second': 552.705, 'eval_steps_per_second': 69.088, 'epoch': 2.0}
{'loss': 0.6298, 'grad_norm': 0.40943172574043274, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.6197056770324707, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.749, 'eval_samples_per_second': 544.714, 'eval_steps_per_second': 68.089, 'epoch': 3.0}
{'loss': 0.6246, 'grad_norm': 0.42999500036239624, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.267973856209150

## BERT Mini

### Full Fine-Tuning

In [ ]:
_ = ft_hyperparameter_tune("prajjwal1/bert-mini", epochs, batch_sizes, learning_rates)

Progress: 1/20


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6088163256645203, 'eval_accuracy': 0.6936274509803921, 'eval_f1': 0.8169838945827232, 'eval_runtime': 0.8245, 'eval_samples_per_second': 494.861, 'eval_steps_per_second': 61.858, 'epoch': 1.0}
{'loss': 0.6267, 'grad_norm': 2.557328939437866, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.5870793461799622, 'eval_accuracy': 0.7009803921568627, 'eval_f1': 0.8157099697885196, 'eval_runtime': 0.8712, 'eval_samples_per_second': 468.311, 'eval_steps_per_second': 58.539, 'epoch': 2.0}
{'loss': 0.5421, 'grad_norm': 13.000743865966797, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.7715953588485718, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.7615526802218114, 'eval_runtime': 1.0786, 'eval_samples_per_second': 378.28, 'eval_steps_per_second': 47.285, 'epoch': 3.0}
{'loss': 0.3946, 'grad_norm': 4.684414386749268, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.2679738562091503}
{'eval_loss': 1.0

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.545035183429718, 'eval_accuracy': 0.7303921568627451, 'eval_f1': 0.819672131147541, 'eval_runtime': 0.8119, 'eval_samples_per_second': 502.497, 'eval_steps_per_second': 62.812, 'epoch': 1.0}
{'loss': 0.584, 'grad_norm': 5.325846195220947, 'learning_rate': 7.276688453159042e-05, 'epoch': 1.0893246187363834}
{'eval_loss': 0.5319600701332092, 'eval_accuracy': 0.7549019607843137, 'eval_f1': 0.8333333333333334, 'eval_runtime': 0.8258, 'eval_samples_per_second': 494.086, 'eval_steps_per_second': 61.761, 'epoch': 2.0}
{'loss': 0.4395, 'grad_norm': 41.37472915649414, 'learning_rate': 4.5533769063180834e-05, 'epoch': 2.178649237472767}
{'eval_loss': 0.769145131111145, 'eval_accuracy': 0.7328431372549019, 'eval_f1': 0.8161888701517707, 'eval_runtime': 0.8119, 'eval_samples_per_second': 502.495, 'eval_steps_per_second': 62.812, 'epoch': 3.0}
{'loss': 0.2954, 'grad_norm': 73.53695678710938, 'learning_rate': 1.8300653594771242e-05, 'epoch': 3.2679738562091503}
{'eval_loss': 0.955430

...

{'eval_loss': 0.578144907951355, 'eval_accuracy': 0.7156862745098039, 'eval_f1': 0.8258258258258259, 'eval_runtime': 0.6484, 'eval_samples_per_second': 629.198, 'eval_steps_per_second': 6.169, 'epoch': 1.0}
{'eval_loss': 0.5186727643013, 'eval_accuracy': 0.7475490196078431, 'eval_f1': 0.8341384863123994, 'eval_runtime': 0.6306, 'eval_samples_per_second': 647.042, 'eval_steps_per_second': 6.344, 'epoch': 2.0}
{'eval_loss': 0.49667444825172424, 'eval_accuracy': 0.7549019607843137, 'eval_f1': 0.8327759197324415, 'eval_runtime': 0.6584, 'eval_samples_per_second': 619.718, 'eval_steps_per_second': 6.076, 'epoch': 3.0}
{'eval_loss': 0.4915316700935364, 'eval_accuracy': 0.7475490196078431, 'eval_f1': 0.8239316239316239, 'eval_runtime': 0.7105, 'eval_samples_per_second': 574.207, 'eval_steps_per_second': 5.629, 'epoch': 4.0}
{'train_runtime': 14.4571, 'train_samples_per_second': 1014.863, 'train_steps_per_second': 8.024, 'train_loss': 0.5085905009302599, 'epoch': 4.0}
{'eval_loss': 0.491531670

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5782576203346252, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8170731707317073, 'eval_runtime': 0.6538, 'eval_samples_per_second': 623.998, 'eval_steps_per_second': 6.118, 'epoch': 1.0}
{'eval_loss': 0.5304329991340637, 'eval_accuracy': 0.7475490196078431, 'eval_f1': 0.8352, 'eval_runtime': 0.6321, 'eval_samples_per_second': 645.431, 'eval_steps_per_second': 6.328, 'epoch': 2.0}
{'eval_loss': 0.5076761245727539, 'eval_accuracy': 0.7598039215686274, 'eval_f1': 0.839344262295082, 'eval_runtime': 0.6595, 'eval_samples_per_second': 618.646, 'eval_steps_per_second': 6.065, 'epoch': 3.0}
{'eval_loss': 0.5055544376373291, 'eval_accuracy': 0.7549019607843137, 'eval_f1': 0.8366013071895425, 'eval_runtime': 0.6424, 'eval_samples_per_second': 635.088, 'eval_steps_per_second': 6.226, 'epoch': 4.0}
{'train_runtime': 14.3729, 'train_samples_per_second': 1020.808, 'train_steps_per_second': 8.071, 'train_loss': 0.54274792506777, 'epoch': 4.0}
{'eval_loss': 0.5055544376373291, 'ev

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5957663059234619, 'eval_accuracy': 0.6985294117647058, 'eval_f1': 0.8183161004431314, 'eval_runtime': 0.6509, 'eval_samples_per_second': 626.864, 'eval_steps_per_second': 6.146, 'epoch': 1.0}
{'eval_loss': 0.5634233355522156, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8163884673748103, 'eval_runtime': 0.6358, 'eval_samples_per_second': 641.721, 'eval_steps_per_second': 6.291, 'epoch': 2.0}
{'eval_loss': 0.5451557040214539, 'eval_accuracy': 0.7352941176470589, 'eval_f1': 0.8301886792452831, 'eval_runtime': 0.6833, 'eval_samples_per_second': 597.089, 'eval_steps_per_second': 5.854, 'epoch': 3.0}
{'eval_loss': 0.5405035614967346, 'eval_accuracy': 0.7450980392156863, 'eval_f1': 0.8354430379746836, 'eval_runtime': 0.636, 'eval_samples_per_second': 641.479, 'eval_steps_per_second': 6.289, 'epoch': 4.0}
{'train_runtime': 14.4151, 'train_samples_per_second': 1017.825, 'train_steps_per_second': 8.047, 'train_loss': 0.5715737836114292, 'epoch': 4.0}
{'eval_loss': 0.5405035

### LoRA

In [ ]:
_ = lora_hyperparameter_tune("prajjwal1/bert-mini", ranks, alphas, epochs, batch_sizes, learning_rates)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Progress: 1/500
{'eval_loss': 0.6154337525367737, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.8525, 'eval_samples_per_second': 478.567, 'eval_steps_per_second': 59.821, 'epoch': 1.0}
{'loss': 0.634, 'grad_norm': 0.4864899814128876, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.5992273092269897, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.8219584569732937, 'eval_runtime': 0.8551, 'eval_samples_per_second': 477.12, 'eval_steps_per_second': 59.64, 'epoch': 2.0}
{'loss': 0.6152, 'grad_norm': 1.3363189697265625, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.5883147120475769, 'eval_accuracy': 0.7107843137254902, 'eval_f1': 0.8201219512195121, 'eval_runtime': 0.9861, 'eval_samples_per_second': 413.733, 'eval_steps_per_second': 51.717, 'epoch': 3.0}
{'loss': 0.5894, 'grad_norm': 2.013890027999878, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.2679738562091503}
{'

## BERT Small

### Full Fine-Tuning

In [ ]:
_ = ft_hyperparameter_tune("prajjwal1/bert-small", epochs, batch_sizes, learning_rates)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Progress: 1/20


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6314049363136292, 'eval_accuracy': 0.678921568627451, 'eval_f1': 0.7937007874015748, 'eval_runtime': 1.0283, 'eval_samples_per_second': 396.766, 'eval_steps_per_second': 49.596, 'epoch': 1.0}
{'loss': 0.648, 'grad_norm': 2.3528854846954346, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.6235907077789307, 'eval_accuracy': 0.6911764705882353, 'eval_f1': 0.8157894736842105, 'eval_runtime': 0.9006, 'eval_samples_per_second': 453.019, 'eval_steps_per_second': 56.627, 'epoch': 2.0}
{'loss': 0.6408, 'grad_norm': 6.450732707977295, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.63517165184021, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.8396, 'eval_samples_per_second': 485.961, 'eval_steps_per_second': 60.745, 'epoch': 3.0}
{'loss': 0.6329, 'grad_norm': 6.159433364868164, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.2679738562091503}
{'eval_loss': 0.6240

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

{'eval_loss': 0.5197935700416565, 'eval_accuracy': 0.7475490196078431, 'eval_f1': 0.8367670364500792, 'eval_runtime': 0.8502, 'eval_samples_per_second': 479.879, 'eval_steps_per_second': 4.705, 'epoch': 1.0}
{'eval_loss': 0.466247022151947, 'eval_accuracy': 0.7867647058823529, 'eval_f1': 0.8589951377633711, 'eval_runtime': 0.8062, 'eval_samples_per_second': 506.059, 'eval_steps_per_second': 4.961, 'epoch': 2.0}
{'eval_loss': 0.4758876860141754, 'eval_accuracy': 0.7843137254901961, 'eval_f1': 0.8562091503267973, 'eval_runtime': 0.8122, 'eval_samples_per_second': 502.36, 'eval_steps_per_second': 4.925, 'epoch': 3.0}
{'eval_loss': 0.489401251077652, 'eval_accuracy': 0.7769607843137255, 'eval_f1': 0.851063829787234, 'eval_runtime': 0.855, 'eval_samples_per_second': 477.168, 'eval_steps_per_second': 4.678, 'epoch': 4.0}
{'train_runtime': 36.2139, 'train_samples_per_second': 405.148, 'train_steps_per_second': 3.203, 'train_loss': 0.41609747656460466, 'epoch': 4.0}
{'eval_loss': 0.48940125107

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5520063042640686, 'eval_accuracy': 0.7083333333333334, 'eval_f1': 0.8166409861325116, 'eval_runtime': 0.8566, 'eval_samples_per_second': 476.274, 'eval_steps_per_second': 4.669, 'epoch': 1.0}
{'eval_loss': 0.5172619819641113, 'eval_accuracy': 0.7524509803921569, 'eval_f1': 0.8404423380726699, 'eval_runtime': 0.9925, 'eval_samples_per_second': 411.071, 'eval_steps_per_second': 4.03, 'epoch': 2.0}
{'eval_loss': 0.5041115880012512, 'eval_accuracy': 0.7475490196078431, 'eval_f1': 0.835725677830941, 'eval_runtime': 0.8093, 'eval_samples_per_second': 504.115, 'eval_steps_per_second': 4.942, 'epoch': 3.0}
{'eval_loss': 0.5032727122306824, 'eval_accuracy': 0.7524509803921569, 'eval_f1': 0.8373590982286635, 'eval_runtime': 0.8324, 'eval_samples_per_second': 490.139, 'eval_steps_per_second': 4.805, 'epoch': 4.0}
{'train_runtime': 36.4974, 'train_samples_per_second': 402.001, 'train_steps_per_second': 3.178, 'train_loss': 0.5086786664765457, 'epoch': 4.0}
{'eval_loss': 0.503272712

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.5684205889701843, 'eval_accuracy': 0.7156862745098039, 'eval_f1': 0.8220858895705522, 'eval_runtime': 0.8242, 'eval_samples_per_second': 495.006, 'eval_steps_per_second': 4.853, 'epoch': 1.0}
{'eval_loss': 0.5529979467391968, 'eval_accuracy': 0.7230392156862745, 'eval_f1': 0.8285280728376327, 'eval_runtime': 0.8791, 'eval_samples_per_second': 464.134, 'eval_steps_per_second': 4.55, 'epoch': 2.0}
{'eval_loss': 0.5299142003059387, 'eval_accuracy': 0.7303921568627451, 'eval_f1': 0.828125, 'eval_runtime': 0.8574, 'eval_samples_per_second': 475.844, 'eval_steps_per_second': 4.665, 'epoch': 3.0}
{'eval_loss': 0.5251005291938782, 'eval_accuracy': 0.7328431372549019, 'eval_f1': 0.8288854003139717, 'eval_runtime': 0.8339, 'eval_samples_per_second': 489.255, 'eval_steps_per_second': 4.797, 'epoch': 4.0}
{'train_runtime': 36.4385, 'train_samples_per_second': 402.651, 'train_steps_per_second': 3.183, 'train_loss': 0.547728209659971, 'epoch': 4.0}
{'eval_loss': 0.5251005291938782, '

### LoRA

In [ ]:
_ = lora_hyperparameter_tune("prajjwal1/bert-small", ranks, alphas, epochs, batch_sizes, learning_rates)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Progress: 1/500
{'eval_loss': 0.5969144105911255, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.8646, 'eval_samples_per_second': 471.912, 'eval_steps_per_second': 58.989, 'epoch': 1.0}
{'loss': 0.623, 'grad_norm': 1.793513298034668, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.5749024152755737, 'eval_accuracy': 0.6985294117647058, 'eval_f1': 0.8177777777777778, 'eval_runtime': 0.9361, 'eval_samples_per_second': 435.838, 'eval_steps_per_second': 54.48, 'epoch': 2.0}
{'loss': 0.5948, 'grad_norm': 4.5539727210998535, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}
{'eval_loss': 0.5578769445419312, 'eval_accuracy': 0.6985294117647058, 'eval_f1': 0.8116385911179173, 'eval_runtime': 0.891, 'eval_samples_per_second': 457.912, 'eval_steps_per_second': 57.239, 'epoch': 3.0}
{'loss': 0.5675, 'grad_norm': 4.53668212890625, 'learning_rate': 5.4901960784313716e-05, 'epoch': 3.2679738562091503}
{'ev